## Import MNIST Images - Deep Learning with PyTorch 14

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

Dataset link: https://www.tensorflow.org/datasets/catalog/mnist

In [5]:
# Convert MNIST image files into Tensor of 4-Dimentions (# of images, Height, Width, Color Channel)
transform = transforms.ToTensor()

In [7]:
#Train Data
train_data = datasets.MNIST(root="cnn_data", train=True, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:04<00:00, 2135104.93it/s]


Extracting cnn_data\MNIST\raw\train-images-idx3-ubyte.gz to cnn_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 101149.89it/s]


Extracting cnn_data\MNIST\raw\train-labels-idx1-ubyte.gz to cnn_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:07<00:00, 232482.81it/s]


Extracting cnn_data\MNIST\raw\t10k-images-idx3-ubyte.gz to cnn_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2271435.41it/s]

Extracting cnn_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to cnn_data\MNIST\raw



In [8]:
#Test Data
test_data = datasets.MNIST(root="cnn_data", train=False, download=True, transform=transform)

In [9]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

Show what directory we are currently in

In [15]:
pwd

'c:\\Learning\\Torch-Tut-Codemy.com'

Show data in the current directory

In [14]:
ls

 Volume in drive C is Windows-SSD
 Volume Serial Number is 42B4-CB90

 Directory of c:\Learning\Torch-Tut-Codemy.com

09/21/2024  08:46 PM    <DIR>          .
09/19/2024  03:17 PM    <DIR>          ..
09/19/2024  03:18 PM    <DIR>          .idea
09/20/2024  11:37 AM            53,837 59954intro_to_CNN.jfif
09/03/2024  11:23 AM             4,551 bezdekIris.data
09/21/2024  08:51 PM             8,329 CNN.ipynb
09/21/2024  08:46 PM    <DIR>          cnn_data
09/21/2024  07:46 PM             3,680 CNN_Intro.ipynb
09/20/2024  12:00 PM            79,414 convolutional-layer.jpg
09/20/2024  11:48 AM            15,114 image.png
09/03/2024  11:23 AM               105 Index
09/03/2024  11:23 AM             4,551 iris.data
09/03/2024  11:23 AM             2,998 iris.names
09/20/2024  12:34 PM            48,869 locally_connected.png
09/20/2024  11:04 AM             3,136 my_iris_torch_model.pt
09/20/2024  11:15 AM            49,716 Part-1-SimpleNN.ipynb
09/21/2024  07:37 PM           168,629 poolin